# Fully connected neural net to qualify wine according to chemical analysis. #

Wine data-set downloaded from [csv-file](http://mng.bz/90Ol)

In [17]:
import pandas as pd
import numpy as np
from pathlib import Path

data_path = Path("./winequality-white.csv")
wine_df = pd.read_csv(data_path, header=0, names=["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", 
"pH", "sulphates", "alcohol", "quality"], sep=";")

# data overview:
#print(wine_df.describe())

# which quality classes do we have? :
qualities = wine_df["quality"].unique()
print(f"Number of unique 'qualities': {len(qualities)}")
print(f"Qualities: {sorted(qualities)}")
binc = np.bincount([q for q in wine_df["quality"]])
no_inst = len(wine_df)
print(f"\nClass counts: {binc}")
print(f"\nNumber of instances: {no_inst} ")
print(f"\nClass fractions: {np.round(binc/no_inst,4) * 100}")


Number of unique 'qualities': 7
Qualities: [3, 4, 5, 6, 7, 8, 9]

Class counts: [   0    0    0   20  163 1457 2198  880  175    5]

Number of instances: 4898 

Class fractions: [ 0.    0.    0.    0.41  3.33 29.75 44.88 17.97  3.57  0.1 ]


In [18]:
import torch
from torch import nn

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class WineNetwork(nn.Module):
    def __init__(self):
        super(WineNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(p=0.2),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 10),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = WineNetwork().to(device)
print(model)

Using cpu device
WineNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=11, out_features=64, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Linear(in_features=128, out_features=256, bias=True)
    (8): ReLU()
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Linear(in_features=256, out_features=10, bias=True)
    (11): ReLU()
  )
)


In [19]:
# define torch.dataset: __init__(), __len__(), __getitem__()
from torch.utils.data import Dataset

class WineDataSet(Dataset):
    def __init__(self, data_df, transform=None, target_transform=None):
        self.data_df = data_df
        self.transform = transform
        self.target_transform = target_transform
        self.X = torch.tensor(self.data_df.iloc[:,:-1].values, dtype=torch.float32)
        self.Y = torch.tensor(self.data_df["quality"].values, dtype= torch.long)

    def __len__(self):
        return len(self.Y)
        
    def __getitem__(self,idx):
        self.x = self.X[idx,:]
        self.y = self.Y[idx]
        if self.transform != None:
            self.x = self.transform(self.x)
        if self.target_transform != None:
            self.y = self.target_transform(self.y)
        return self.x, self.y
            

In [20]:
from sklearn.preprocessing import StandardScaler

def normalize_dataframe(data_df, column_names_to_normalize):
    """
        Normalizes all given columns of a given data frame with a StandardScaler from Sklearn. 
        Input:
            data_df: dataframe with numerical values to normalize
            column_names_to_normalize: list of the names of the columns to be normalized
        Output:
            dataframe with columns normalized
    """
    scaler = StandardScaler()
    data_to_norm = data_df[column_names_to_normalize].values
    data_normed = scaler.fit_transform(data_to_norm)
    df_temp = pd.DataFrame(data_normed, columns=column_names_to_normalize, index=data_df.index)
    data_df[column_names_to_normalize]= df_temp
    return data_df


In [21]:
# IS PROBABLY NOT NEEDED - SEE PYTORCH'S DEFINITION OF CROSSENTROPYLOSS:

# OHE encoding of the labels: 
labels = np.sort(wine_df["quality"].unique())
ohe=torch.nn.functional.one_hot(torch.tensor(labels))

# make a target_transform for the dataloaders:
target_transform = {l:oh for l,oh in zip(labels,ohe)}.get

# test the target_transform:
target_transform(4)

tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [22]:
# test and train loops:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

def train_loop(dataloader, model, loss_fn, optimizer):
    losses, no_correct = 0, 0
    for n_batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        losses += loss.item()
        no_correct +=(pred.argmax(1)==y).sum().item()

        optimizer.zero_grad()        
        loss.backward()
        optimizer.step()
    
    return losses, no_correct
        

def test_loop(dataloader, model, loss_fn):
    losses, no_correct = 0, 0
    with torch.no_grad():
        for (X,y) in dataloader:
            pred = model(X)
            losses += loss_fn(pred, y).item()
            no_correct += (pred.argmax(1)== y).sum().item()
     
    return losses, no_correct
    

In [55]:
#a = torch.randn(3,5, requires_grad=False)
#b = a.softmax(dim=1)
#b, b.argmax(1)
(b.argmax(1)==torch.tensor([0,2,2], requires_grad=False)).sum().item()

2

In [ ]:
# Train the model:
from torch.utils.data import DataLoader
import os
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split

# writer for tensorboard:
writer = SummaryWriter()

# create new model instance:
model = WineNetwork().to(device) 

# loss function:
# cross-entropy:
loss_fn = nn.CrossEntropyLoss()

# optimizer:
# adam:
# optimizer_name = "ADAM"
#learning_rate = 1e-4
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# sgd:
optimizer_name = "SGD"
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)

# training parameters:
from pathlib import Path
data_path = Path("./winequality-white.csv")
column_names = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", 
"pH", "sulphates", "alcohol", "quality"]
column_names_to_normalize = column_names[:-1]
wine_df = pd.read_csv(data_path, header=0, names=column_names, sep=";")


epochs = 400
write_log_after_epochs = 50

epoch_losses = []
epoch_no_corrects = []
train_losses = []
train_acc = []
test_losses = []
test_acc = []

test_size=0.2
train_df, test_df = train_test_split(wine_df, test_size=test_size)

train_df = normalize_dataframe(train_df, column_names_to_normalize)
test_df = normalize_dataframe(test_df, column_names_to_normalize)
train_ds = WineDataSet(data_df=train_df)
test_ds = WineDataSet(data_df=test_df)

batch_size=64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

best_model_name = ""
max_correct = -torch.inf

for ep in range(1, epochs+1):        
       
        # put model in train mode:
        model.train()
        (train_loss, train_no_correct) = train_loop(train_dl, model, loss_fn, optimizer)
              
        # switch model to to evaluation mode:
        model.eval()
        (test_loss, test_no_correct) = test_loop(test_dl, model, loss_fn)

        if(test_no_correct > max_correct):
            max_correct = test_no_correct
            if(best_model_name):
                os.remove(best_model_name)
            best_model_name = "./model_" + str(test_no_correct) + "_" + str(learning_rate) + "_" + str(ep) + "_" + str(batch_size) + "_" + optimizer_name + ".pt"
            torch.save(model.state_dict(), best_model_name)

        writer.add_scalar("Loss/test", test_loss/ len(test_ds), ep)
        writer.add_scalar("Accuracy/test", test_no_correct/ len(test_ds), ep)
        writer.add_scalar("Loss/train", train_loss/ len(train_ds), global_step=ep)
        writer.add_scalar("Accuracy/train", train_no_correct/ len(train_ds), global_step=ep)
       
        if ep % write_log_after_epochs == 0:
            print(f"\n----- Epoch: {ep} -----")
            print(f"Epoch loss: {test_loss/ len(test_ds)}")
            print(f"Epoch accuracy: {test_no_correct/ len(test_ds)}")
            

### Try and Improve - best Model ###

model: model_640_0.001_369_64_SGD.pt <br>
<br>
Training parameters:  <br>
optimizer_name = "SGD" <br>
learning_rate = 1e-3 <br>
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9) <br>
epochs = 400 <br>
test_size=0.2 <br>
batch_size=64 <br>


In [34]:
640/980

0.6530612244897959

In [ ]:
# THIS TRAINING & EVALUATION CONTAINS A SERIOUS ERROR:
# - RANDOM SPLITTING IN EVERY EPOCH EXPOSES ALL DATA TO THE TRAINING AFTER SOME EPOCHS!!!



# Train the model:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split, StratifiedKFold

# writer for tensorboard:
writer = SummaryWriter()

# create new model instance:
model = WineNetwork().to(device) 

# loss function:
# cross-entropy:
loss_fn = nn.CrossEntropyLoss()

# optimizer:
# adam:
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# sgd:
#learning_rate = 1e-5
#optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)

# training parameters:
from pathlib import Path
data_path = Path("./winequality-white.csv")
column_names = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", 
"pH", "sulphates", "alcohol", "quality"]
column_names_to_normalize = column_names[:-1]
wine_df = pd.read_csv(data_path, header=0, names=column_names, sep=";")

apply_stratified_kfold = True
no_folds = 5

epochs = 200
write_log_after_epochs = 100

epoch_losses = []
epoch_no_corrects = []
train_losses = []
train_acc = []
test_losses = []
test_acc = []
for ep in range(1, epochs+1):

    if apply_stratified_kfold:
        X = wine_df[column_names_to_normalize]
        y = wine_df["quality"].values
        skf = StratifiedKFold(n_splits=no_folds, shuffle=True, random_state=42)
        losses, acc = 0, 0

        for fold, (train_ids, test_ids) in enumerate(skf.split(X, y)):
            train_df = wine_df.iloc[train_ids].copy()
            train_df = normalize_dataframe(train_df, column_names_to_normalize)
            test_df = wine_df.iloc[test_ids].copy()
            test_df = normalize_dataframe(test_df, column_names_to_normalize)
            train_ds = WineDataSet(data_df=train_df, target_transform=target_transform)
            test_ds = WineDataSet(data_df=test_df, target_transform=target_transform)
            train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
            test_dl = DataLoader(test_ds, batch_size=32, shuffle=True)

            # put model in train mode:
            model.train()
            (train_loss, train_no_correct) = train_loop(train_dl, model, loss_fn, optimizer)
            if ep % write_log_after_epochs == 0:
                train_losses.append(train_loss/ len(train_ds))
                train_acc.append(train_no_correct/ len(train_ds))

            # switch model to to evaluation mode:
            model.eval()
            (test_loss, test_no_correct) = test_loop(test_dl, model, loss_fn)
            if ep % write_log_after_epochs == 0:
                test_losses.append(test_loss/len(test_ds))
                test_acc.append(test_no_correct/ len(test_ds))
                    
        if ep % write_log_after_epochs == 0:
            print(f"\n----- Epoch: {ep} -----")
            print(f"Epoch loss (avg. all folds): {np.average(test_losses)}")
            print(f"Epoch accuracy (avg. all folds): {np.average(test_acc)}")
            writer.add_scalar("Loss/train", np.average(train_losses), ep)
            writer.add_scalar("Accuracy/train", np.average(train_acc), ep)
            writer.add_scalar("Loss/test", np.average(test_losses), ep)
            writer.add_scalar("Accuracy/test", np.average(test_acc), ep)
            train_losses.clear()
            test_losses.clear()
            train_acc.clear()
            test_acc.clear()
    
    else:
        # THIS IS WRONG - RANDOM SPLITTING IN EVERY EPOCH EXPOSES ALL DATA TO THE TRAINING AFTER SOME EPOCHS!!!
        train_df, test_df = train_test_split(wine_df, test_size=0.2)
        train_df = train_df.copy()
        test_df = test_df.copy()
        train_df = normalize_dataframe(train_df, column_names_to_normalize)
        test_df = normalize_dataframe(test_df, column_names_to_normalize)
        train_ds = WineDataSet(data_df=train_df, target_transform=target_transform)
        test_ds = WineDataSet(data_df=test_df, target_transform=target_transform)
        train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
        test_dl = DataLoader(test_ds, batch_size=32, shuffle=True)
       
        # put model in train mode:
        model.train()
        (train_loss, train_no_correct) = train_loop(train_dl, model, loss_fn, optimizer)
              
        # switch model to to evaluation mode:
        model.eval()
        (test_loss, test_no_correct) = test_loop(test_dl, model, loss_fn)
       
        if ep % write_log_after_epochs == 0:
            print(f"\n----- Epoch: {ep} -----")
            print(f"Epoch loss: {test_loss/ len(test_ds)}")
            print(f"Epoch accuracy: {test_no_correct/ len(test_ds)}")
            writer.add_scalar("Loss/test", test_loss/ len(test_ds), ep)
            writer.add_scalar("Accuracy/test", test_no_correct/ len(test_ds), ep)
            writer.add_scalar("Loss/train", train_loss/ len(train_ds), global_step=ep)
            writer.add_scalar("Accuracy/train", train_no_correct/ len(train_ds), global_step=ep)
        
writer.flush()
writer.close()
print("----- Finished Training -----")

In [ ]:
# Train the model:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split, StratifiedKFold

# writer for tensorboard:
writer = SummaryWriter()

# create new model instance:
model = WineNetwork().to(device) 

# loss function:
# cross-entropy:
loss_fn = nn.CrossEntropyLoss()

# optimizer:
# adam:
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# sgd:
#learning_rate = 1e-5
#optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)

# training parameters:
from pathlib import Path
data_path = Path("./winequality-white.csv")
column_names = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", 
"pH", "sulphates", "alcohol", "quality"]
column_names_to_normalize = column_names[:-1]
wine_df = pd.read_csv(data_path, header=0, names=column_names, sep=";")

apply_stratified_kfold = True
no_folds = 5

epochs = 200
write_log_after_epochs = 100

epoch_losses = []
epoch_no_corrects = []
train_losses = []
train_acc = []
test_losses = []
test_acc = []

train_df, test_df = train_test_split(wine_df, test_size=0.2)
train_df = train_df.copy()
test_df = test_df.copy()
train_df = normalize_dataframe(train_df, column_names_to_normalize)
test_df = normalize_dataframe(test_df, column_names_to_normalize)
# OHE encoding is not necessary for Pytorch's Crossentropyloss
#train_ds = WineDataSet(data_df=train_df, target_transform=target_transform)
#test_ds = WineDataSet(data_df=test_df, target_transform=target_transform)
train_ds = WineDataSet(data_df=train_df)
test_ds = WineDataSet(data_df=test_df)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=True)


for ep in range(1, epochs+1):        
       
        # put model in train mode:
        model.train()
        (train_loss, train_no_correct) = train_loop(train_dl, model, loss_fn, optimizer)
              
        # switch model to to evaluation mode:
        model.eval()
        (test_loss, test_no_correct) = test_loop(test_dl, model, loss_fn)
       
        if ep % write_log_after_epochs == 0:
            print(f"\n----- Epoch: {ep} -----")
            print(f"Epoch loss: {test_loss/ len(test_ds)}")
            print(f"Epoch accuracy: {test_no_correct/ len(test_ds)}")
            writer.add_scalar("Loss/test", test_loss/ len(test_ds), ep)
            writer.add_scalar("Accuracy/test", test_no_correct/ len(test_ds), ep)
            writer.add_scalar("Loss/train", train_loss/ len(train_ds), global_step=ep)
            writer.add_scalar("Accuracy/train", train_no_correct/ len(train_ds), global_step=ep)